In [164]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
matplotlib.use('TkAgg')

In [165]:
data_path = '/Users/aboba/Downloads/B_l.dat'
data = np.loadtxt(data_path, usecols=(0, 1, 2))

In [166]:
x_data, y_data, z_data = data[:, 0], data[:, 1], data[:, 2]


In [167]:
def f0(x, y): return 1
def f1(x, y): return 2
def f2(x, y): return -0.5 * y**2 + x**2
def f3(x, y): return -1.5 * x * y**2 + x**3
def f4(x, y): return 3/8 * y**4 - 3 * y**2 * x**2 + x**4
def f5(x, y): return 15/8 * x * y**4 - 5 * x**3 * y**2 + x**5

In [168]:
# Функция создания модели
def create_F2_model(functions):
    def model(xy, *coefficients):
        x, y = xy
        result = 0
        for f, a in zip(functions, coefficients):
            result += a * f(x, y)
        return result
    return model

In [169]:
# Выбор функций и начальных предположений для коэффициентов
functions = [f0,f2,f3,f4]
initial_guess = [1,1,1,1]  # 0, A0, A2, A3


In [170]:
# Создание модели F2
F2_model = create_F2_model(functions)

# Выполнение подгонки
popt, pcov = curve_fit(F2_model, (x_data, y_data), z_data, p0=initial_guess, maxfev=50000)

# Результаты подгонки и их стандартные отклонения
coefficients_fit = popt
errors = np.sqrt(np.diag(pcov))

# Форматированный вывод результатов подгонки и их отклонений в процентах
for i, (coef, error) in enumerate(zip(coefficients_fit, errors)):
      if i<1:
        percent_error = (error / coef * 100) if coef != 0 else float('inf')
        print(f"A{i} = {coef} ± {error} ({percent_error:.2f}%)")
      else:   
        percent_error = (error / coef * 100) if coef != 0 else float('inf')
        print(f"A{i+1} = {coef} ± {error} ({percent_error:.2f}%)")

A0 = 0.3752032615249104 ± 0.0005299122903364798 (0.14%)
A2 = -0.0012496927231682685 ± 4.2440539109202796e-05 (-3.40%)
A3 = 0.00026520125556468043 ± 7.980598880556098e-06 (3.01%)
A4 = -1.4801210960627932e-05 ± 4.966905996713293e-07 (-3.36%)


In [171]:
# Визуализация результатов
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Точки данных
ax.scatter(x_data, y_data, z_data, color='b', label='Исходные данные')

# Подогнанная поверхность
x_range = np.linspace(min(x_data), max(x_data), 30)
y_range = np.linspace(min(y_data), max(y_data), 30)
x_grid, y_grid = np.meshgrid(x_range, y_range)
z_fit = F2_model((x_grid, y_grid), *coefficients_fit)

ax.plot_surface(x_grid, y_grid, z_fit, color='r', alpha=0.5, label='Подогнанная модель')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.legend()
plt.show()
